# <center>CMPBIO210, IB120, IB201
# <center>"Introduction to Quantitative Methods in Biology"
# <center>Lecture 3. Numerical Methods of Solving ODEs
## <center>Denis Titov

**The goal of this jupyter notebook is to introduce you to how numerical methods for solution of ordinary differential equations (ODEs) work in general and review various option of SciPy solve_ivp() function. This is a math heavy subject but it's good to have a basic idea of how ODE solvers like solve_ivp() actually work if you plan to use them.**

In [ ]:
import sympy as sym
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Example: Clearance of drugs from the blood

We will use drug clearance from the blood through kidney filtration as an example in this section.
Drug clearance from the blood can be describe using differential equation $\frac{d[Drug](t)}{dt}=-k[Drug](t)$, where $t$ is time,  k is some constant describing kindey filtration rate,  $[Drug](t)$ is blood concentration of drug at time $t$ and $[Drug]_0$ is is blood concentration of drug at $t=0$. 

# Numerical solution of $\frac{d[Drug]}{dt}=-k[Drug]$

#### We can solve $\frac{d[Drug](t)}{dt}=-k[Drug](t)$ numerically using solve_ivp() from SciPy library.
To solve this differential equation numerically means to find a bunch of numbers that correspond to $[Drug](t)$ starting from some initial condition, e.g. $[Drug](0)=1$

In [ ]:
k = 1
Drug_0 = [1]
tspan = [0, 5]

def ODE(t, y):
    dDrug_dt = -k * y
    return dDrug_dt

soln = solve_ivp(ODE, tspan, Drug_0, t_eval=np.linspace(0, 5, 1000))

plt.plot(soln.t, soln.y[0], label="solve_ivp() solution")
plt.legend();

*solve_ivp()* has many different methods to numerically solve differential equations with specific methods being good for specific equations. You can see the list of methods by running *solve_ivp?* command

Now, let's write code to solve $\frac{d[Drug]}{dt}=-k[Drug]$ without using *solve_ivp()* function with the goal of better understanding how *solve_ivp()* and similar methods actually works.

## Euler method

The basic idea behind numerical methods for solving ODEs is to simply take some small number for $\Delta t$ and do repetitive algebraic calculation with it.  
As an example, let's take some value of $\Delta t$ and find a solution for $\frac{d[Drug](t)}{dt}=-k[Drug](t)$ starting from initial $[Drug]_0$  
  
First, we rewrite $\frac{d[Drug](t)}{dt}=-k[Drug](t)$ as $\frac{\Delta[Drug]}{\Delta t}=\frac{[Drug]_{t+\Delta t}-[Drug]_{t}}{\Delta t}=-k[Drug]_{t}$.  
  
Rearranging the latter we get an equation to calculate $[Drug]_{t+\Delta t}$ from $[Drug]_t$:  
$[Drug]_{t+\Delta t}=[Drug]_t-\Delta t \cdot k \cdot [Drug]_t$  
  
Using the latter formula, we can now use our initial condition $[Drug]_{t=0}=[Drug]_0$ to find $[Drug]_{\Delta t}$:  
  
$[Drug]_{\Delta t}=[Drug]_0-\Delta t \cdot k \cdot [Drug]_0=[Drug]_0 \cdot (1-\Delta t \cdot k)$  
  
Repeat the same to get $[Drug]_{2 \cdot \Delta t}$, $[Drug]_{3 \cdot \Delta t}$ ... etc:  
  
$[Drug]_{2 \cdot \Delta t}=[Drug]_{\Delta t}-\Delta t \cdot k \cdot [Drug]_{\Delta t}=[Drug]_0 \cdot (1-\Delta t \cdot k)^2$  
  
$[Drug]_{3 \cdot \Delta t}=[Drug]_0 \cdot (1-\Delta t \cdot k)^3$... etc etc etc for as many $\Delta t$ steps as we want.   
  
This is called the explicit Euler method and is the simplest method that can be used to numerically solve ODEs although it is rarely used in practice as there are better methods as we will review below.  

### Let's implement Euler method with python code.

#### For loops

We will need to use a *for loop* to implement Euler method.  
*For loops* are very common in computer programs and is a way to make computer do a repetitive task like Euler method.  
Here's a simple example of a *for loop*:

In [ ]:
for i in range(5):
    print(i)

Note that numbering in python starts at 0 as opposed to 1 and that last number (i.e. 5 above) is not included.  
*Side note: this is a major point of contention betweem programming languages with more math/science focused languages having 1-based indexing (e.g, Fortran, Julia, Matlab, R) while other languages made a poor choice of having 0 based indexing which is in my humble opinion nonsensical! (e.g., C, Python, Java)*

#### Euler method code

In [ ]:
dt = 0.5
k = 1

Drug = np.array([1,])
time = np.array([0,])
for t in range(20):
    Drug = np.append(Drug, Drug[t - 1] - dt * k * Drug[t - 1])
    time = np.append(time, time[t - 1] + dt)

plt.scatter(time, Drug, label='Euler')
plt.legend();

Note that you have to choose $\Delta t$ carefully as $\Delta t$>2 will produce an unstable solution because if $|1-\Delta t \cdot k|>1$ than numerical solution will go to $\infty$ if number of steps goes to $\infty$.  

## Why do people continue to invent other methods if Euler works?
There're hundreds of numerical methods that people have invented.  
Numerical methods have to balance numerical accuracy vs computational speed.  
Euler method will be 10x more accurate if you decrease $\Delta t$ 10x but it will also take 10x longer for computer to calculate a solution because it'll have to take 10x more steps. Because of this tradeoff between speed and accuracy, people have spent lots of time inventing new methods that can be as accurate as Euler while taking much larger steps or much faster than Euler with the same accuracy.  More modern methods are orders of magnitude better than Euler as we'll se below.

#### $\frac{d[Drug]}{dt}=-k[Drug]$ is simple enough so that we can also solve it analytically using dsolve() function from SymPy library:

In [ ]:
Drug = sym.symbols("[Drug]", cls=sym.Function)
t, k, Drug_0 = sym.symbols("t, k, [Drug]_0")
sym.dsolve(sym.diff(Drug(t), t) + k * Drug(t), Drug(t), ics={Drug(0): Drug_0})

#### Let's compare Euler method with analytical solution and solve_ivp() solution

In [ ]:
dt = 0.5
k = 1

Drug = np.array([1,])
time = np.array([0,])
for t in range(1, round(5/dt)+1):
    Drug = np.append(Drug, Drug[t - 1] - dt * k * Drug[t - 1])
    time = np.append(time, time[t - 1] + dt)

Drug_0 = [1]
tspan = [0, 5]
def fun(t, y):
    dDrug_dt = -k * y
    return dDrug_dt

soln = solve_ivp(fun, tspan, Drug_0)
plt.scatter(time, Drug, label='Euler', color="red")
plt.scatter(soln.t, soln.y[0], label="solve_ivp() solution", color="green")
timepoints_for_analyt_sol = np.linspace(tspan[0], tspan[1],1000)
plt.plot(timepoints_for_analyt_sol, np.exp(-k*timepoints_for_analyt_sol), label='[Drug]=exp(-kt)', color="grey")
plt.legend();

Curves for solve_ivp() and analytical solution are indistinguishable above and Euler at very low $\Delta t$ also merges with them

### Source of errors in Euler and other numerical methods

All numerical methods are approximations of the analytical solution and the error of numerical methods is the difference between numerical solution and analytical solution.  
  
  Let's analyze the error introduced by Euler method above and find how error depends on $\Delta t$.

**Taylor Series overview**  
One way to analyze the magnitude of error introduced by Euler method is by using Taylor series.  
You likely learned about Taylor series in intro calculus: https://en.wikipedia.org/wiki/Taylor_series.   
  
  Taylor series is the following equation and is key for many numerical ODE methods:  
  
  $y(t+\Delta t)=y(t)+\Delta t \frac{dy(t)}{dt} + \frac {\Delta t^2}{2} \frac{d^2y(t)}{dt^2} + ... + \frac{\Delta t^n}{n!} \frac{d^ny(t)}{dt^n}$  
  
  Taylor series is more and more accurate at higher $n$ all the way to $n \rightarrow \infty$
  
  

**Lets understand source of error of Euler method using Taylor series**  
  In our case, Taylor series will look like this:  
  
  $[Drug]_{t+\Delta t}=[Drug]_{t}+\Delta t \frac{d[Drug]}{dt} + \frac {\Delta t^2}{2} \frac{d^2[Drug]}{dt^2} + ... + \frac{\Delta t^n}{n!} \frac{d^n[Drug]}{dt^n}$  
  
  Since we know that $\frac{d[Drug]}{dt}=-k[Drug]$ and $k=1$:  
  
  $[Drug]_{t+\Delta t}=[Drug]_{t}-\Delta t [Drug]_{t} + \frac {\Delta t^2}{2} [Drug]_{t} + ... + \frac{\Delta t^n}{n!} (-1)^n[Drug]_{t}$  
  
Note that Euler method *truncates* the Taylor series at the second component so the error of Euler method is  equal to  
(read about $O()$ notation here: https://en.wikipedia.org/wiki/Big_O_notation)  
  
  $O(\Delta t^2)=\frac {\Delta t^2}{2} [Drug]_{t} + ... + \frac{\Delta t^n}{n!}(-1)^n[Drug]_{t}$  
  
  and the equation is often written as:  
  
$[Drug]_{t+\Delta t}=[Drug]_{t}-\Delta t [Drug]_{t} + O(\Delta t^2)$  
  
  So the error per step is proportinal to $\Delta t^2$ and assuming errors are cummulative the total error after time $t$ is proportinal to $N_{Steps} \cdot \Delta t^2 \sim \Delta t$ because $N_{Steps} \sim \frac{1}{\Delta t}$.  
  So error of Euler method at time $t$ is proportional to $\Delta t$.

## Taylor method

  Let's see if we can improve Euler method if we truncate at third or fourth term of Taylor series:

In [ ]:
dt = 0.5
k = 1

Drug = [1,]
DrugTaylor2 = [1,]
DrugTaylor3 = [1,]
time = [0,]

for t in range(1, round(5/dt)+1):
    Drug = np.append(Drug, Drug[t - 1] - dt * k * Drug[t - 1])
    DrugTaylor2 = np.append(
        DrugTaylor2, 
        (DrugTaylor2[t - 1] 
         - dt * k * DrugTaylor2[t - 1] 
         + DrugTaylor2[t - 1]/2*(dt*k)**2)
    )
    DrugTaylor3 = np.append(
        DrugTaylor3, 
        (DrugTaylor3[t - 1] 
         - dt * k * DrugTaylor3[t - 1] 
         + DrugTaylor3[t - 1]/2*(dt*k)**2 
         - DrugTaylor3[t - 1]/6*(dt*k)**3)
    )
    time = np.append(time, time[t - 1] + dt)

plt.scatter(time, Drug,label='Euler')
plt.scatter(time, DrugTaylor2,label='Taylor $O(\Delta t)^2$')
plt.scatter(time, DrugTaylor3,label='Taylor $O(\Delta t)^3$')
timepoints_for_analyt_sol = np.linspace(tspan[0], tspan[1],1000)
plt.plot(timepoints_for_analyt_sol, np.exp(-k*timepoints_for_analyt_sol),label='[Drug]=exp(-kt)', color="grey")
plt.legend();

This method is also rarely used in practice as it can be time consuming to compute $\frac{d^ny(t)}{dt^n}$ for $n>1$.  
We of course always know $\frac{d^ny(t)}{dt^n}$ for $n=1$ because that's the right hand side of a differential equation.  
  
  The errors per step of Taylor methods with one or two additional Taylor series terms compared to Euler are proportinal to $\Delta t^3$ and $\Delta t^4$ and total error after time $t$ is proportional to $N_{Steps} \cdot \Delta t^3 \sim \Delta t^2$ and $N_{Steps} \cdot \Delta t^4 \sim \Delta t^3$ because $N_{Steps} \sim \frac{1}{\Delta t}$.  
  
  Error of Taylor methods with one or two additional Taylor series terms at time $t$ is proportional to $\Delta t^2$ and $\Delta t^3$, respectively.  
  
  For example, for $\Delta t=0.1$ Taylor methods will be 10x and 100x more accurate!

## Heun's method / Runge-Kutta Method

The basic idea behind Heun method is to improve Euler method by using the information from the next timepoint to correct the solution.

Let's use the same ODE $\frac{d[Drug](t)}{dt}=-k[Drug](t)$ starting from initial $[Drug]_0$  
  
Euler method is:  
$[Drug]_{t+\Delta t}=[Drug]_t-\Delta t \cdot k \cdot [Drug]_t$  

Heun method is:  
$\widetilde{[Drug]}_{t+\Delta t}=[Drug]_t-\Delta t \cdot k \cdot [Drug]_t$  
$[Drug]_{t+\Delta t}=[Drug]_t-\frac{\Delta t}{2} \cdot k ([Drug]_t + \widetilde{[Drug]}_{t+\Delta t})$ 

Heun method takes a step $\Delta t$ multiplied by an average of derivative of current time point $t$ (i.e., $-k \cdot [Drug]_t$) and derivative at next timepoint $t+\Delta t$ (i.e., $-k \cdot \widetilde{[Drug]}_{t+\Delta t}$) instead of just a derivative at current tipmeoint $t$ as in Euler method. It thus improves on the accuracy of Euler method by using a corrected / better estimate of where the solution should be going. A good way to understand Heun method is to use geometric interpretation of derivatives as slopes as is done here: https://en.wikipedia.org/wiki/Heun%27s_method

In [ ]:
Drug = [1,]
DrugTaylor2 = [1,]
Drug_Heun = [1,]
time = [0,]
dt = 0.5
k = 1
for t in range(1, round(5/dt)+1):
    Drug = np.append(Drug, Drug[t - 1] - dt * k * Drug[t - 1])
    Drug_Heun_guess = Drug_Heun[t - 1] - dt * k * Drug_Heun[t - 1]
    Drug_Heun = np.append(
        Drug_Heun, 
        (Drug_Heun[t - 1] 
         - (1/2) * dt * k * (Drug_Heun[t - 1] + Drug_Heun_guess))
    )
    DrugTaylor2 = np.append(
        DrugTaylor2, 
        (DrugTaylor2[t - 1] 
         - dt * k * DrugTaylor2[t - 1] 
         + DrugTaylor2[t - 1]/2*(dt*k)**2)
    )
    time = np.append(time, time[t - 1] + dt)

plt.scatter(time, Drug, color='red',label='Euler')
plt.scatter(time, Drug_Heun, color='green',label='Heun')
plt.scatter(time, DrugTaylor2, color='purple',label='Taylor $O(\Delta t)^2$', marker='.')
timepoints_for_analyt_sol = np.linspace(tspan[0], tspan[1],1000)
plt.plot(timepoints_for_analyt_sol, np.exp(-k*timepoints_for_analyt_sol),label='[Drug]=exp(-kt)', color="grey")
plt.legend();

## Adaptive timesteps

Methods used by *solve_ivp()* also implement adaptive timestepping where the method can basically adjust $\Delta t$ to be bigger or smaller to match the user defined accuracy of the solution defined using *tol* keyword in *solve_ivp()*.  
  
Adaptive timestepping is evident in the code below if you note that the distance between points of the solution are not evenly spaced if we make *solve_ivp()* to plot only the points it used for getting the solution by not including *t_eval* keyword argument.

In [ ]:
k = 1
Drug_0 = [1]
tspan = [0, 5]

def fun(t, y):
    dDrug_dt = -k * y
    return dDrug_dt

soln = solve_ivp(fun, tspan, Drug_0, method='RK45', atol = 1e-6, rtol = 1e-3)

plt.scatter(soln.t, soln.y[0], label="solve_ivp() solution")
plt.legend();

Adaptive timestepping is implemented by using two numerical method with different accuracy (e.g., Euler and Heun) and using method with higher accuracy as solution and using difference between two methods to estimate error and then increase/decrease $\Delta t$ if error is below/above the user defined *tol*.

## Better methods to numerically solve ODEs are continuing to be developed

Heun method and related Runge-Kutta methods were developed in early 1900s!   

Many more methods have been described (and are continuing to be described) to numerically solve differential equations including higher order Runge-Kutta methods as well as completely different methods. 
  
You can take whole courses describing various numerical methods which are beyond the scope of this one lecture but the principle is similar to Euler, Taylor and Heun methods described above.

# SciPy solve_ivp()

The great thing about SciPy *solve_ivp()* is that you don't have to implement complex numerical algorithms yourself as we did above because *solve_ivp()* already has many algorithms implemented and you can just use them without fully understanding how they work but hopefully you have some idea from discussion above.